In [30]:
### Installing packages
# pip install selenium
# pip install webdrive_manager
# pip install yfinance

# Import packages
import yfinance as yf
import requests

from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

### Strategy

The stock screening strategy below is designed to complement the VWAP and ABCD pattern day trading strategies outlined in the book "Day Trading for a Living" by

#### Pre-Market Gappers
Criteria for the Gappers Scanner includes:
- Stocks that in the pre-market gapped up or down at least 2%
- Stocks that have traded at least 100,000 shares by 9am in pre-market
- Stocks that have an average daily volume of over 500,000 shares
- Stocks that have Average True Range (ATR) of at least 50 cents
- There is a fundamental catalyst for the stock

#### Additional Filter
We will filter for only Medium Float stocks (20 to 500 million shares) with a price range between $10-$100 and Large float stocks (500+ million shares) with a price range > $20. These stock work best with the VWAP and ABCD pattern trading strategies.

#### Human Screen
I will review the output manually and do my own research into whether there is a fundamental catalyst for the stock


In [31]:
# Initialising Selenium and setting options
options = webdriver.ChromeOptions() 
options.headless = True 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) 

In [32]:
# Lets begin by webscraping a list of the top 100 losers and top 100 gainers for the day

# Gainers
t100_gainers_url = "https://finance.yahoo.com/screener/predefined/day_gainers?count=100&offset=0"

# Losers
t100_losers_url = "https://finance.yahoo.com/screener/predefined/day_losers?count=100&offset=0"

In [33]:
# Function to scrape list stock symbols from Yahoo Finance's screener list
def scrape_gappers(url, driver):
    # Load page
	driver.get(url)
    
	# Find row element
	row_eles = driver.find_elements(By.CLASS_NAME, 'simpTblRow')

	print(row_eles)

	# In each row we are interested in the Ticker Symbol and Price. 
	# Although we will also be filtering for Volume and Float (Market Cap/Price), we will do this at a later stage once we search for the ticker symbols using the yfinance library. This will save us the trouble of having to parse text with B and M representing billions and millions.
	gappers_list = list(map(row_to_tuple, row_eles))
	return gappers_list

# Function to convert Yahoo Finance row element to a tuple of (Ticker Symbol, Price)
def row_to_tuple(row_ele):
	# Getting the Ticker Symbol
	symbol_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Symbol"]')
	symbol_str = symbol_ele.text

	# Getting the Price
	price_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Price (Intraday)"]')
	price_num = float(price_ele.text)

	# Returning data as a tuple - (Ticker Symbol: String, Price: Float)
	return (symbol_str, price_num)

In [34]:
# Stock ticker symbols
t100_gainers = scrape_gappers(t100_gainers_url, driver)
# t100_losers = scrape_gappers(t100_losers_url, driver)

print(t100_gainers)

#


[<selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="d3d2f3a8-03b2-492c-8bb7-1cd0bfdcf20d")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="01fc0831-da8d-431a-8f1a-6bebd1169e01")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="4bd24a11-32f0-4e1b-8bf5-5ad06176986f")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="ef7a2bdb-1436-44be-92f5-92f755e4f018")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="5acf3acc-96c0-4201-88e8-6f1c6cf3a463")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="f8c83cb9-39ea-4ef3-b38b-96cd6c209c89")>, <selenium.webdriver.remote.webelement.WebElement (session="fa9bb58cebd5c70c324aeaafa084eaa1", element="15099c1c-a983-4344-b3b3-0cee36aa